## Importação de Bibliotecas Utilizadas

In [ ]:
#Importação de bibliotecas necessárias 
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import os
#KNN
from sklearn.neighbors import NearestNeighbors
#Cosine Similarity
from sklearn.decomposition import TruncatedSVD
import sklearn.metrics.pairwise as pw
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

## Variáveis / Constantes

In [ ]:
# Pasta de entrada para arquivos tratados
input_path = "../data/datasets_pos_tratamento"

#caminho/nomeDoArquivo
fileRatings = os.path.join(input_path, "ratings_tratados.parquet")
#caminho/nomeDoArquivo
fileMeta = os.path.join(input_path, "meta_tratados.parquet")

# Quantidade mínima de reviews de um usuário a ser considerado
qtdMinReviewsPerUser = 4

# Quantidade mínima de reviews de um produto a ser considerado
qtdMinReviewsPerProduct = 10

## Leitura e verificação dos Datasets (Análise Exploratória)

In [3]:
#Leitura de dataset 1 (Avaliações)

dfRatings = pd.read_parquet(fileRatings, engine="pyarrow")
#headers das colunas
print(dfRatings.columns)
#tamanho (linhas, colunas)
dfRatings.shape

Index(['ID_USUARIO', 'IDENTIFICADOR', 'AVALIACAO', 'ANO', 'MES', 'DIA',
       'DIA_DA_SEMANA'],
      dtype='object')


(2920862, 7)

In [4]:
dfRatings.info()
dfRatings.value_counts('IDENTIFICADOR')

<class 'pandas.core.frame.DataFrame'>
Index: 2920862 entries, 1900398 to 2219110
Data columns (total 7 columns):
 #   Column         Dtype  
---  ------         -----  
 0   ID_USUARIO     object 
 1   IDENTIFICADOR  object 
 2   AVALIACAO      float32
 3   ANO            int32  
 4   MES            int32  
 5   DIA            int32  
 6   DIA_DA_SEMANA  object 
dtypes: float32(1), int32(3), object(3)
memory usage: 133.7+ MB


IDENTIFICADOR
B0C1G1BJ2B    24069
B0B53DWRVW    23525
B07F4P3JH7    20512
B00U8QEXBS    12830
B017UQCB3A    12399
              ...  
B09J2BRDPY        1
B09J2G3Z39        1
0742551253        1
B09J2HYY2C        1
987835816X        1
Name: count, Length: 70962, dtype: int64

In [ ]:
#Exibe as linhas iniciais
dfRatings.head()

,ID_USUARIO,IDENTIFICADOR,AVALIACAO,ANO,MES,DIA,DIA_DA_SEMANA
1900398,AG7ZXMLZ2YZ2I3VVIYOYIAE6HF5Q,B09RNB69M1,5.0,2017,3,18,Sábado
2601334,AHOZBDJBNNV7FVSRQ2GAAU2LWQAA,B01FVU86JA,5.0,2016,8,16,Terça-feira
811553,AGGK4QPNMSBZ6V3OLDZRS6T6F5AA,B071K5BQPF,5.0,2021,3,9,Terça-feira
2493452,AFY5OL7TTL7WXTH7B3R64TNH24IQ,B017UQCB3A,5.0,2023,2,9,Quinta-feira
388891,AE7BEGTB54W7PGWE4Y35BRLNHMWQ,B0B7K4FBS8,5.0,2022,10,3,Segunda-feira
2290082,AGJCG7RJUUQ3XGEE3ZMSAHWDVHXA,B08N9VRR6H,5.0,2021,8,30,Segunda-feira
1693169,AEM2SR7W5GCUIQM764FVOUSQ3KKQ,B09STDSC89,5.0,2015,5,3,Domingo
2305513,AF4EGW7GMTAI3EU6KPVN3O4MWPRQ,B07N7Q8YDD,5.0,2019,1,4,Sexta-feira
1820531,AFEX2TVVVHEHUCDDGE37GCGD2RZQ,B07FPWNHMN,5.0,2018,3,12,Segunda-feira
1280543,AEVA46SHYZV2M5GBPDCAUBHVOFNA,B01LOMZ8ZY,5.0,2015,8,23,Domingo


In [7]:
#Exibe as linhas finais
dfRatings.tail()

,ID_USUARIO,IDENTIFICADOR,AVALIACAO,ANO,MES,DIA,DIA_DA_SEMANA
1692743,AGECB2FB7SVF2ZKXISGR6URGZMYA,B0995BDSRW,5.0,2022,1,26,Quarta-feira
2356330,AECEHKSB5CIGB2UH2PW3LL5DLDSQ,B0987L1TCY,1.0,2022,4,3,Domingo
2229084,AEH6JKECCSTGJTQC5TONNVMELDNQ,B0BFXX2V21,1.0,2019,1,7,Segunda-feira
2768307,AHTVCWTKTD6Y2YYLYLZGSFAEMH5Q,B08R826WX3,4.0,2021,7,27,Terça-feira
2219110,AG4LHCTYX2TQUTILSUKR6SDOAYDA,B07F1HTGX6,5.0,2020,9,2,Quarta-feira


In [ ]:
#Leitura de dataset 2 (Informações)

#caminho/nomeDoArquivo
dfMeta = pd.read_parquet(fileMeta, engine='pyarrow')
#headers das colunas
print(dfMeta.columns) 

#tamanho (linhas, colunas)
dfMeta.shape

Index(['IDENTIFICADOR', 'TITULO', 'MEDIA_VOTOS', 'QTD_AVALIACOES',
       'CARACTERISTICAS', 'DESCRICAO', 'PRECO', 'LOJA', 'DETALHES',
       'CATEGORIAS'],
      dtype='object')


(75693, 10)

In [9]:
#Exibe as linhas iniciais
dfMeta.head()

,IDENTIFICADOR,TITULO,MEDIA_VOTOS,QTD_AVALIACOES,CARACTERISTICAS,DESCRICAO,PRECO,LOJA,DETALHES,CATEGORIAS
52011,B08V9799ZG,"Electric Can Opener, Restaurant Can Opener, Fu...",2.7,23,,,15.040000,W-Dragon,"Date First Available, Color, Material, Brand, ...","Home & Kitchen, Kitchen & Dining, Kitchen Uten..."
67694,B09JGJ7D1S,"PAPAISON Roller Skates for Women and Men, Delu...",4.6,720,【RETRO STYLE 】 Classic high top double-row rol...,,79.989998,PAPAISON,"Size, Color, Brand, Wheel Material, Wheel Type...","Sports & Outdoors, Sports, Skates, Skateboards..."
3050,B06XB6LF6Y,Dewhel Sport Front Bumper Tow Hook License Pla...,3.4,166,"High quality aluminum Tow hook License Plate,B...",Tow Hook License Plate Fitments:2000-2009 Hond...,20.990000,DEWHEL,"Manufacturer, Brand, Item Weight, Package Dime...","Automotive, Exterior Accessories, License Plat..."
66635,B07RSM5PRD,JR Studio 3x5 inch Yellow Combat Vet Tab Shape...,4.2,13,Size: Check Item title - Professionally made i...,,4.990000,JR Studio,"Brand, Color, Room Type, Recommended Uses For ...","Sports & Outdoors, Fan Shop, Auto Accessories,..."
64560,B08KXF2S4V,"Horsemen's Pride Horse Feed Tub, Small, 18 Qua...",4.4,36,This small/mini feed tub measures 14 inches ac...,The Small Feed Tub measures 14 inches across (...,32.590000,Jolly Pets,"Product Dimensions, Item model number, Date Fi...","Pet Supplies, Horses, Stable Supplies, Feeding..."


In [10]:
#Exibe as linhas iniciais
dfMeta.tail()

,IDENTIFICADOR,TITULO,MEDIA_VOTOS,QTD_AVALIACOES,CARACTERISTICAS,DESCRICAO,PRECO,LOJA,DETALHES,CATEGORIAS
37194,B082B8WPW5,"THRRLY Airpods Case, Girls Cute Clear Airpods ...",4.3,85,【Compatibility】This case designed for Airpods ...,"Compatibity:, Compatible with Apple Airpods 1s...",11.980000,THRRLY,"Package Dimensions, Item Weight, Manufacturer,...","Electronics, Headphones, Earbuds & Accessories..."
6265,B09G6QB78V,MAYASAF EGR Valve EGV598 1998 99 2000 01 02 03...,4.1,13,【Part Fitment】For 1998-2004 Pathfinder;Frontie...,,44.990002,MAYASAF,"Manufacturer, Part Number, Item Weight, Packag...","Automotive, Replacement Parts, Exhaust & Emiss..."
54886,B08LXSPQZC,Avana Chaise Lounge Chair for Stretching and R...,4.2,52,COMFY & UNIQUE -- Comfortable and unique style...,,439.000000,Avana,"Color, Brand, Product Dimensions, Style, Speci...","Home & Kitchen, Furniture, Living Room Furnitu..."
860,B08PMQJRYQ,NIXON District Beanie,4.2,11,"100% Acrylic, Made in USA or Imported, Machine...",The primary materials that compose this produc...,16.000000,NIXON,"Department, Date First Available, Manufacturer",AMAZON FASHION
15795,1101908211,Poems from Greek Antiquity (Everyman's Library...,4.7,40,A beautiful Pocket Poet selection of short poe...,"About the Author, PAUL QUARRIE, author of, Win...",15.950000,Paul Quarrie (Editor),"Publisher, Language, Hardcover, ISBN 10, ISBN ...","Books, Literature & Fiction, Poetry"


In [11]:
dfMeta[dfMeta['IDENTIFICADOR'] == 'B08B8L36LD']

,IDENTIFICADOR,TITULO,MEDIA_VOTOS,QTD_AVALIACOES,CARACTERISTICAS,DESCRICAO,PRECO,LOJA,DETALHES,CATEGORIAS
8638,B08B8L36LD,Upgraded Kids Travel Tray with Dry Erase Top C...,4.2,3021,【Upgraded Mess-Free Transparent Dry Erase Top】...,,33.970001,tomser,"Specific Uses For Product, Special Feature, Co...","Baby Products, Car Seats & Accessories, Access..."


## Pré Processamento dos Dados

### Seleção de colunas utilizadas

In [12]:
#Arquivo Meta

#Selecionando as colunas utilizada
dfMeta = dfMeta[['IDENTIFICADOR', 'TITULO']]

dfMeta.head()

,IDENTIFICADOR,TITULO
52011,B08V9799ZG,"Electric Can Opener, Restaurant Can Opener, Fu..."
67694,B09JGJ7D1S,"PAPAISON Roller Skates for Women and Men, Delu..."
3050,B06XB6LF6Y,Dewhel Sport Front Bumper Tow Hook License Pla...
66635,B07RSM5PRD,JR Studio 3x5 inch Yellow Combat Vet Tab Shape...
64560,B08KXF2S4V,"Horsemen's Pride Horse Feed Tub, Small, 18 Qua..."


In [13]:
#Arquivo Avaliações

#Selecionando as colunas utilizada
dfRatings = dfRatings[['ID_USUARIO', 'IDENTIFICADOR', 'AVALIACAO']]

dfRatings.head()

,ID_USUARIO,IDENTIFICADOR,AVALIACAO
1900398,AG7ZXMLZ2YZ2I3VVIYOYIAE6HF5Q,B09RNB69M1,5.0
2601334,AHOZBDJBNNV7FVSRQ2GAAU2LWQAA,B01FVU86JA,5.0
811553,AGGK4QPNMSBZ6V3OLDZRS6T6F5AA,B071K5BQPF,5.0
2493452,AFY5OL7TTL7WXTH7B3R64TNH24IQ,B017UQCB3A,5.0
388891,AE7BEGTB54W7PGWE4Y35BRLNHMWQ,B0B7K4FBS8,5.0


### Junção dos dados (avaliações e produto)

In [16]:
# Unindo as colunas
dfMerged = dfRatings.merge(dfMeta, on = 'IDENTIFICADOR')
dfMerged.head(20)

,ID_USUARIO,IDENTIFICADOR,AVALIACAO,TITULO
0,AG7ZXMLZ2YZ2I3VVIYOYIAE6HF5Q,B09RNB69M1,5.0,Gorilla Grip Anti Fatigue Cushioned Kitchen Fl...
1,AHOZBDJBNNV7FVSRQ2GAAU2LWQAA,B01FVU86JA,5.0,A&R Sports Blade Cover
2,AGGK4QPNMSBZ6V3OLDZRS6T6F5AA,B071K5BQPF,5.0,GOER 3.2 ft x 9.8 ft Metallic Tinsel Foil Frin...
3,AFY5OL7TTL7WXTH7B3R64TNH24IQ,B017UQCB3A,5.0,"GoTags Stainless Steel Pet ID Tags, Personaliz..."
4,AE7BEGTB54W7PGWE4Y35BRLNHMWQ,B0B7K4FBS8,5.0,Cooking for One: A My Bizzy Kitchen Cookbook
5,AGJCG7RJUUQ3XGEE3ZMSAHWDVHXA,B08N9VRR6H,5.0,Westport Red Porch Rocker Cushions - Standard ...
6,AEM2SR7W5GCUIQM764FVOUSQ3KKQ,B09STDSC89,5.0,Lawrence Frames 766081 Nutmeg Wood Picture Fra...
7,AF4EGW7GMTAI3EU6KPVN3O4MWPRQ,B07N7Q8YDD,5.0,SpaGuard Spa Shock-Oxidizer (35 oz) 2-Pack
8,AFEX2TVVVHEHUCDDGE37GCGD2RZQ,B07FPWNHMN,5.0,YOUR SMILE Pure Blue Square Decorative Throw P...
9,AEVA46SHYZV2M5GBPDCAUBHVOFNA,B01LOMZ8ZY,5.0,"Edens Garden Sniffles & Sneezes ""OK for Kids"" ..."


In [18]:
# Verificando o tamanho do novo df
dfMerged.shape

(2920862, 4)

In [19]:
# Verificando se ele possui valores nulos
dfMerged.isna().sum()

ID_USUARIO       0
IDENTIFICADOR    0
AVALIACAO        0
TITULO           0
dtype: int64

In [20]:
# Descartando valores duplicados (mesmo usuário avaliando o mesmo produto)
dfMerged.drop_duplicates(['ID_USUARIO', 'IDENTIFICADOR'], keep='last', inplace=True)

In [21]:
# Verificando o tamanho após remoção de duplicatas
dfMerged.shape

(2873379, 4)

In [22]:
# Descartando a variável utilizada para mesclagem (não mais necessária)
del dfMerged['IDENTIFICADOR']

#Visualizando o novo df 
dfMerged.head(20)

,ID_USUARIO,AVALIACAO,TITULO
0,AG7ZXMLZ2YZ2I3VVIYOYIAE6HF5Q,5.0,Gorilla Grip Anti Fatigue Cushioned Kitchen Fl...
2,AGGK4QPNMSBZ6V3OLDZRS6T6F5AA,5.0,GOER 3.2 ft x 9.8 ft Metallic Tinsel Foil Frin...
3,AFY5OL7TTL7WXTH7B3R64TNH24IQ,5.0,"GoTags Stainless Steel Pet ID Tags, Personaliz..."
4,AE7BEGTB54W7PGWE4Y35BRLNHMWQ,5.0,Cooking for One: A My Bizzy Kitchen Cookbook
5,AGJCG7RJUUQ3XGEE3ZMSAHWDVHXA,5.0,Westport Red Porch Rocker Cushions - Standard ...
6,AEM2SR7W5GCUIQM764FVOUSQ3KKQ,5.0,Lawrence Frames 766081 Nutmeg Wood Picture Fra...
7,AF4EGW7GMTAI3EU6KPVN3O4MWPRQ,5.0,SpaGuard Spa Shock-Oxidizer (35 oz) 2-Pack
8,AFEX2TVVVHEHUCDDGE37GCGD2RZQ,5.0,YOUR SMILE Pure Blue Square Decorative Throw P...
10,AGUWZQSX67A2YT6ZSWSJVJHD5O6Q,5.0,"Flents Wipe 'N Clear Lens Cleaning Wipes 5"" x ..."
11,AHQIXNAW7SDAM3CXXLGE5CNV3ECA,5.0,"Sonix Clear Phone Case for iPhone 13 Pro, Drop..."


### Filtrando dados pós mesclagem

In [23]:
# Removendo produtos e usuários com poucas avaliações no dataframe final.

# Filtrar usuários com poucas avaliações
user_counts = dfMerged['ID_USUARIO'].value_counts()
valid_users = user_counts[user_counts >= qtdMinReviewsPerUser].index
dfMerged = dfMerged[dfMerged['ID_USUARIO'].isin(valid_users)]

# Filtrar produtos com poucas avaliações
product_counts = dfMerged['TITULO'].value_counts()
valid_products = product_counts[product_counts >= qtdMinReviewsPerProduct].index
dfMerged = dfMerged[dfMerged['TITULO'].isin(valid_products)]

In [24]:
dfMerged.shape

(281364, 3)

### Pivoteamento

In [44]:
# Realizando o pivoteamento
pivot = dfMerged.pivot_table(columns = 'ID_USUARIO', index = 'TITULO', values = 'AVALIACAO')

In [46]:
# Verificando o arquivo transformado em PIVOT
pivot.head()

ID_USUARIO,AE225JSR3YVYTIDQSVM4P5OIPOCQ,AE225YRHUP7CFMWEUCXWFPINWC5Q,AE225Z2VRWT6GPTOMA4H4O3H2KVQ,AE226IHCMYPYN4AIZKNRVH7ETC4A,AE227AY2ASJOP4KGTHAPXJPJZBTQ,AE22BT6AWP25OAIPYV36NWGXMYPA,AE22EBF2Z3HHH2NIDGWE74GDGY7A,AE22EMKSL7H7LR6RRQRULU6GROPA,AE22EOOOTMJOODRKZEOCVDODY4GA,AE22IAKPVLI327DHYMXQ3O5OI6GQ,...,AHZZE5FFZHW7Z7MNQPW3BWZSBQ2A,AHZZGVTSABWNT5GASPL4EUMUUO3Q,AHZZIGH3VTUACFSUSVQL3W3QDS5A,AHZZJQYNVZUJNPNQ737ITGEQUB4A,AHZZKDIRNXTRU252MOZB5FQBNF5Q,AHZZL323ORE6AHMIB2OCLYNCRXJQ,AHZZM2KGP52A6KBAS2IZ2RVZUJJA,AHZZPUYPNZQ7QXK55HGGE3Z7TTEA,AHZZSZREMJDFLOOOFH4ZA2VBXOZA,AHZZVAZOOI2VLLRPY44BRT66RAVA
TITULO,,,,,,,,,,,,,,,,,,,,,
'47 MLB Unisex-Adult Men's Clean Up Cap,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'47 NFL Adult Men's Beanie Knit Hat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'47 NFL Men's Basic Scrum Tee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(2 Pack) Supershieldz Designed for Samsung (Galaxy J7 Star) Tempered Glass Screen Protector, Anti Scratch, Bubble Free",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(2 in 1) Tensea for Waterproof Apple Watch Screen Protector Case SE 2022 Series SE 6 5 4 40mm Accessories, iWatch Protective PC Face Cover Built-in Tempered Glass Film, Front & Back Bumper Women Men",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Função de busca de títulos 

In [34]:
def searchProductName(product_name, quantity=10):
    """
    Função para buscar produtos pelo nome e mostrar os primeiros resultados.
    """
    
    if product_name is None or product_name.strip() == "":
        print("Forneça um nome de produto válido.")
        return
    
    # Buscar títulos únicos que contenham o termo pesquisado
    mask = dfMerged['TITULO'].str.lower().str.contains(product_name.lower())
    unique_titles = dfMerged.loc[mask, 'TITULO'].unique()
    if unique_titles.size > 0:
        for title in unique_titles[:quantity]:
            print(title)
    else:
        print("Nenhum produto encontrado.")


## Aplicação do Cosine Similarity para recomendação a partir de produtos

#### Tratamento e treino

In [26]:
pivot_filled = pivot.fillna(0)

In [31]:
# Calcular a matriz de similaridade baseada nos produtos
item_similarity_df = pd.DataFrame(
    pw.cosine_similarity(pivot_filled), 
    index=pivot_filled.index,
    columns=pivot_filled.index
)

item_similarity_df.head()

TITULO,'47 MLB Unisex-Adult Men's Clean Up Cap,'47 NFL Adult Men's Beanie Knit Hat,'47 NFL Men's Basic Scrum Tee,"(2 Pack) Supershieldz Designed for Samsung (Galaxy J7 Star) Tempered Glass Screen Protector, Anti Scratch, Bubble Free","(2 in 1) Tensea for Waterproof Apple Watch Screen Protector Case SE 2022 Series SE 6 5 4 40mm Accessories, iWatch Protective PC Face Cover Built-in Tempered Glass Film, Front & Back Bumper Women Men","(Old Model) Toshiba 3TB SATA 6Gb/s 7200rpm 3.5"" Inte","(Pack of 2) Universal 14 Inches High Performance Electric Slim Radiator Cooling Fan with Mounting Kit, 12V, Blue","(Packaging May Vary) Earth Rated Compostable Dog Poop Bags, Ultra Thick and Leak Proof, Unscented, 225 Count",(with Secure Lock) Armor Case Cover for Samsung Galaxy Buds2 Pro Case(2022) /Galaxy Buds Pro Case(2021) /Galaxy Buds 2 Case (2021) /Galaxy Buds Live Case(2020) with Keychain/Zipper Box/Brush -Purple,0.5 Liter Bottled Water,...,"winees Baby Monitor, 1080p Indoor Secuirty Camera Wi-Fi Smartphone Pet Cam, Motion Detection, 2-Way Audio, 2.4GHZ, Night Vision, Multi Installation IP Cam, Work with Alexa (Mini)","wooDsom Powerful Magnetic Knife Strip, Holder Made in USA (Alder, 12 inches)","wookon Stackable Storage Bins with Lids-76 Qt Office Organization,Collapsible Storage Bins,Plastic Storage Bins with Wheels,4 Packs Closet Organizers and Storage,Grey Folding storage box,File Cabinet","yoerm Fake Plants Artificial Bonsai Tree for Office Wall Book Shelf Room Decor, Tall 9.5""","yourose Satin Pillowcase for Hair and Skin, 2 Pack King Size Silky Pillowcases with Envelop Closure, (Navy, 20”X40”,2pcs)",yueton Pack of 10 Wire Lead Battery Storage Box Case Holder for 18650 Button Top Single Battery,"· Petgrow · Artificial Grass Turf Lawn 7FTX15FT,Economy Indoor Outdoor Synthetic Grass Mat 0.4inch Pile Height, Backyard Patio Garden Balcony Rug, Rubber Backing/Drainage Holes,Customized Sizes","【2021 Upgraded】 Western Home Round Dog Bed for Small Dogs, Calming Donut Cuddler Pet Bed,Fluffy Plush Faux Fur Cat Bed(20"", Grey)",﻿﻿VEVA Air Purifier Filter Replacements - 6 Pack Precut & Compatible with Honey-well HPA200 Purifiers - Premium Activated Carbon Pre-Filters,"𝗪𝗜𝗡𝗡𝗘𝗥 - 𝗕𝗘𝗦𝗧 𝗞𝗘𝗧𝗢 𝗖𝗥𝗘𝗔𝗠𝗘𝗥 by LevelUp® with C8 MCT, Collagen Protein, Ketogenic Diet Coffee Creamer, Ketosis Supplement Ketone Support (Original Flavor, Unsweetened 19 oz)"
TITULO,,,,,,,,,,,,,,,,,,,,,
'47 MLB Unisex-Adult Men's Clean Up Cap,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'47 NFL Adult Men's Beanie Knit Hat,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'47 NFL Men's Basic Scrum Tee,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(2 Pack) Supershieldz Designed for Samsung (Galaxy J7 Star) Tempered Glass Screen Protector, Anti Scratch, Bubble Free",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(2 in 1) Tensea for Waterproof Apple Watch Screen Protector Case SE 2022 Series SE 6 5 4 40mm Accessories, iWatch Protective PC Face Cover Built-in Tempered Glass Film, Front & Back Bumper Women Men",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Criação da máquina preditiva

In [ ]:
# Função de recomendação
def get_item_recommendations(item_id, item_similarity_df, num_recommendations=5):
    
    # Verificar se o item está no DataFrame de similaridade
    if item_id not in item_similarity_df.index:
        raise KeyError(f"O item '{item_id}' não foi encontrado no item_similarity_df.")
    
    # Obter as similaridades do item com todos os outros
    similar_items = item_similarity_df[item_id].sort_values(ascending=False)

    # Remover o próprio item da lista
    similar_items = similar_items.drop(item_id, errors='ignore')
    
    # Remover itens com similaridade 0.0
    similar_items = similar_items[similar_items != 0.0]
    
    # Retornar os itens mais similares (apenas os top N)
    return similar_items.head(num_recommendations)

In [36]:
faixas = pd.cut((pivot_filled > 0).sum(axis=1), bins=range(0, (pivot_filled > 0).sum(axis=1).max() + 6, 5))
contagem = faixas.value_counts().sort_index()
print(contagem)
print(pivot_filled.shape[0])

(0, 5]             0
(5, 10]          489
(10, 15]        1655
(15, 20]         881
(20, 25]         593
                ... 
(3290, 3295]       0
(3295, 3300]       0
(3300, 3305]       0
(3305, 3310]       0
(3310, 3315]       1
Name: count, Length: 663, dtype: int64
6086


In [36]:
def recommendationByProductSimilarity(product, num_of_elements=4):
    """
    Função para recomendar produtos similares com base na similaridade de itens.
    """
    try:
        
        if product is None or product.strip() == "":
            raise ValueError("Forneça um nome de produto válido.")
        
        recommendations = get_item_recommendations(product, item_similarity_df, num_recommendations=num_of_elements)
        
        if len(recommendations) < num_of_elements:
            print(f"Aviso: Menos de {num_of_elements} recomendações disponíveis para o produto\n")
            num_of_elements = len(recommendations)
            
        print(f"Produto escolhido: '{product}'\n")
        print(f"Top {num_of_elements} recomendações:")
        for item in recommendations.index:
            print(item)
    except KeyError as e:
        print(e)

#### Funcionamento

In [61]:
searchProductName(input('Pesquise aqui o produto:'), 10)

Pesquise aqui o produto: couch


JUSPURBET Decorative Velvet Throw Pillows Covers for Couch Bed Sofa,Pack of 2 Soild Soft Cushion Cases,26x26 Inches,Black
NANPIPER Fleece Blanket Sherpa Throw,Soft Fuzzy Twin Blankets for Couch,Orange 60"x80"
Foamily Throw Pillows Insert 12 x 12 Inches - Bed and Couch Decorative Pillow - Made in USA - Bed and Couch Sham Filler
Pack of 2 CaliTime Cozy Fleece Throw Pillow Cases Covers for Couch Bed Sofa Christmas Snowflakes Both Sides 12 X 20 Inches Sea Blue
MIULEE Pack of 2 Luxury Faux Fur Fluffy Throw Pillow Covers Set Soft Deluxe Decorative Plush Fleece Pillowcases for Cushion Couch Sofa Bedroom Home 16 x 16 Inch Burgundy
Fancy Homi Pack of 2 Tan Boho Decorative Throw Pillow Covers 18x18 with Pom-poms, Soft Corduroy Solid Square Cushion Cases Set for Living Room Couch Sofa Bedroom Car (18x18 Inch/45x45 cm, Khaki)
Utopia Bedding Throw Pillows Insert (Pack of 2, Navy) - 12 x 20 Inches Bed and Couch Pillows - Indoor Decorative Pillows
Bedsure Sherpa Fleece Throw Blanket for Couch - Thick

In [66]:
recommendationByProductSimilarity(input('Entre com um produto que tenha gostado: '), 10) 

Entre com um produto que tenha gostado:  JUSPURBET Decorative Velvet Throw Pillows Covers for Couch Bed Sofa,Pack of 2 Soild Soft Cushion Cases,26x26 Inches,Black


Produto escolhido: 'JUSPURBET Decorative Velvet Throw Pillows Covers for Couch Bed Sofa,Pack of 2 Soild Soft Cushion Cases,26x26 Inches,Black'

Top 10 recomendações:
WLNUI Set of 2 Soft Velvet Sage Green Pillow Covers 18x18 Inch Square Decorative Throw Pillow Covers Cushion Case for Sofa Couch Home Farmhouse Decor
Extra Thick Mattress Topper Full Size, Pillow Top Full Mattress Pad Topper, Plush Quilted Fitted Full Thick Mattress Cover with Soft Down Alternative Fill, 6-21 Inch Deep Pocket
MIULEE Pack of 2 Velvet Pillow Covers Fall Autumn Decorative Square Pillowcase Soft Solid Cushion Case for Sofa Bedroom Car 14 x 14 Inch Orange
W-RARA Beaded Bracelet Compatible with Fitbit Versa 2/Fitbit Versa/Fitbit Versa Lite Bands for Women, Stretchy Nylon Solo Loop Strap Elastic Handmade Boho Cute Stylish Wristbands for Fitbit Versa Smart Watch
Tom Clancy Enemy Contact (A Jack Ryan Jr. Novel Book 6)
MERNETTE New Year/Christmas Decorations Velvet Soft Decorative Square Throw Pillow Cover Cushion C

## Aplicação do Cosine Similarity para recomendação a partir de usuário (SVD)

#### Tratamento e treino

In [22]:
pivot_filled = pivot.fillna(0)
pivot_filled.head(10)

ID_USUARIO,AE225JSR3YVYTIDQSVM4P5OIPOCQ,AE225YRHUP7CFMWEUCXWFPINWC5Q,AE225Z2VRWT6GPTOMA4H4O3H2KVQ,AE226IHCMYPYN4AIZKNRVH7ETC4A,AE227AY2ASJOP4KGTHAPXJPJZBTQ,AE22BT6AWP25OAIPYV36NWGXMYPA,AE22EBF2Z3HHH2NIDGWE74GDGY7A,AE22EMKSL7H7LR6RRQRULU6GROPA,AE22EOOOTMJOODRKZEOCVDODY4GA,AE22IAKPVLI327DHYMXQ3O5OI6GQ,...,AHZZE5FFZHW7Z7MNQPW3BWZSBQ2A,AHZZGVTSABWNT5GASPL4EUMUUO3Q,AHZZIGH3VTUACFSUSVQL3W3QDS5A,AHZZJQYNVZUJNPNQ737ITGEQUB4A,AHZZKDIRNXTRU252MOZB5FQBNF5Q,AHZZL323ORE6AHMIB2OCLYNCRXJQ,AHZZM2KGP52A6KBAS2IZ2RVZUJJA,AHZZPUYPNZQ7QXK55HGGE3Z7TTEA,AHZZSZREMJDFLOOOFH4ZA2VBXOZA,AHZZVAZOOI2VLLRPY44BRT66RAVA
TITULO,,,,,,,,,,,,,,,,,,,,,
'47 MLB Unisex-Adult Men's Clean Up Cap,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'47 NFL Adult Men's Beanie Knit Hat,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'47 NFL Men's Basic Scrum Tee,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(2 Pack) Supershieldz Designed for Samsung (Galaxy J7 Star) Tempered Glass Screen Protector, Anti Scratch, Bubble Free",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(2 in 1) Tensea for Waterproof Apple Watch Screen Protector Case SE 2022 Series SE 6 5 4 40mm Accessories, iWatch Protective PC Face Cover Built-in Tempered Glass Film, Front & Back Bumper Women Men",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(Old Model) Toshiba 3TB SATA 6Gb/s 7200rpm 3.5"" Inte",5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(Pack of 2) Universal 14 Inches High Performance Electric Slim Radiator Cooling Fan with Mounting Kit, 12V, Blue",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(Packaging May Vary) Earth Rated Compostable Dog Poop Bags, Ultra Thick and Leak Proof, Unscented, 225 Count",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
(with Secure Lock) Armor Case Cover for Samsung Galaxy Buds2 Pro Case(2022) /Galaxy Buds Pro Case(2021) /Galaxy Buds 2 Case (2021) /Galaxy Buds Live Case(2020) with Keychain/Zipper Box/Brush -Purple,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
# # Criando o modelo SVD com um número de componentes que atinjas os 80%

svd = TruncatedSVD(n_components=1700, random_state=42)
svd.fit(pivot_filled)

# # Descomentar para ver a variância explicada acumulada
# explained = np.cumsum(svd.explained_variance_ratio_)

# plt.figure(figsize=(10, 6))
# plt.plot(explained, marker='o')
# plt.xlabel('Número de componentes')
# plt.ylabel('Variância explicada acumulada')
# plt.title('Escolha de n_components com base na variância explicada')
# plt.grid(True)
# plt.axhline(y=0.8, color='r', linestyle='--')  
# plt.show()

TruncatedSVD(n_components=1700, random_state=42)

In [24]:
latent_matrix = svd.fit_transform(pivot_filled)

# Reconstruir a matriz aproximada
reconstructed_matrix = np.dot(latent_matrix, svd.components_)
reconstructed_df = pd.DataFrame(reconstructed_matrix, index=pivot_filled.index, columns=pivot_filled.columns)

print("Matriz reconstruída:")
reconstructed_df.head(5)

Matriz reconstruída:


ID_USUARIO,AE225JSR3YVYTIDQSVM4P5OIPOCQ,AE225YRHUP7CFMWEUCXWFPINWC5Q,AE225Z2VRWT6GPTOMA4H4O3H2KVQ,AE226IHCMYPYN4AIZKNRVH7ETC4A,AE227AY2ASJOP4KGTHAPXJPJZBTQ,AE22BT6AWP25OAIPYV36NWGXMYPA,AE22EBF2Z3HHH2NIDGWE74GDGY7A,AE22EMKSL7H7LR6RRQRULU6GROPA,AE22EOOOTMJOODRKZEOCVDODY4GA,AE22IAKPVLI327DHYMXQ3O5OI6GQ,...,AHZZE5FFZHW7Z7MNQPW3BWZSBQ2A,AHZZGVTSABWNT5GASPL4EUMUUO3Q,AHZZIGH3VTUACFSUSVQL3W3QDS5A,AHZZJQYNVZUJNPNQ737ITGEQUB4A,AHZZKDIRNXTRU252MOZB5FQBNF5Q,AHZZL323ORE6AHMIB2OCLYNCRXJQ,AHZZM2KGP52A6KBAS2IZ2RVZUJJA,AHZZPUYPNZQ7QXK55HGGE3Z7TTEA,AHZZSZREMJDFLOOOFH4ZA2VBXOZA,AHZZVAZOOI2VLLRPY44BRT66RAVA
TITULO,,,,,,,,,,,,,,,,,,,,,
'47 MLB Unisex-Adult Men's Clean Up Cap,-0.004833,-0.007240,0.001856,0.002014,-0.007543,0.001050,-0.005289,0.002544,0.000499,-0.005801,...,0.001714,0.004706,-0.001329,0.000426,-0.004480,0.000641,-0.007734,-0.003060,-0.002264,-0.012676
'47 NFL Adult Men's Beanie Knit Hat,0.001558,-0.011097,-0.002339,0.002902,-0.003726,0.000645,-0.001397,-0.000065,-0.001110,0.000666,...,0.002439,-0.003135,-0.000941,-0.005347,-0.000279,0.000204,0.001575,-0.007103,-0.010505,-0.002386
'47 NFL Men's Basic Scrum Tee,0.002989,0.016441,-0.000287,0.017148,0.009539,0.007534,0.012370,0.017687,-0.000979,-0.003209,...,-0.003054,-0.000778,-0.000657,0.000679,0.050371,0.006559,-0.007465,0.000881,-0.000466,-0.019589
"(2 Pack) Supershieldz Designed for Samsung (Galaxy J7 Star) Tempered Glass Screen Protector, Anti Scratch, Bubble Free",-0.003436,-0.010233,-0.004307,0.001876,-0.009008,-0.002627,-0.001949,0.000647,0.000042,-0.002237,...,0.008405,-0.009631,-0.001812,-0.004327,-0.008265,0.001690,0.007877,-0.001209,0.002371,-0.005288
"(2 in 1) Tensea for Waterproof Apple Watch Screen Protector Case SE 2022 Series SE 6 5 4 40mm Accessories, iWatch Protective PC Face Cover Built-in Tempered Glass Film, Front & Back Bumper Women Men",-0.001774,-0.011134,-0.001622,-0.003655,0.003689,-0.002875,-0.005383,-0.003844,0.000305,-0.000377,...,-0.002817,0.001692,-0.001018,-0.002471,-0.025682,-0.001498,-0.003528,-0.015411,-0.004066,-0.002012


#### Criação da máquina preditiva

In [ ]:
# 4. Função para recomendar produtos (baseado em SVD reconstruído)
def recommendationByUserSVD(user_id, num_recommendations=4):
    
    try:
        
        if user_id is None or user_id.strip() == "":
            raise ValueError("Forneça um ID de usuário válido.")
        
        if user_id not in pivot_filled.columns:
            print(f"Usuário '{user_id}' não encontrado.")
            return
        
        # Ratings originais e previstos do usuário
        user_ratings = pivot[user_id]
        predicted_ratings = reconstructed_df[user_id]

        # Identificar produtos não avaliados pelo usuário
        not_rated = user_ratings[user_ratings.isna()].index

        # Filtrar previsões apenas para produtos não avaliados
        recommendations = predicted_ratings[not_rated].sort_values(ascending=False).head(num_recommendations)

        print(f"Recomendações para o usuário '{user_id}'\n")
        print(f"Top {num_recommendations} recomendações:")
        for product in recommendations.index:
            print(product)
    except KeyError as e:
        print(e)

#### Funcionamento

In [28]:
recommendationByUserSVD(input("Entre com o ID do Usuário desejado"), 10)  

Entre com o ID do Usuário desejado AHZZM2KGP52A6KBAS2IZ2RVZUJJA


Recomendações para o usuário 'AHZZM2KGP52A6KBAS2IZ2RVZUJJA'

Top 10 recomendações:
KESSAKU Chef Knife - 8 inch - Dynasty Series - Razor Sharp Kitchen Knife - Forged ThyssenKrupp German High Carbon Stainless Steel - G10 Garolite Handle with Blade Guard
SUPERIOR Jacquard Matelasse Fleur De Lis 100% Cotton Medallion 3-Piece Bedspread Set - Twin, Silver
Kutting Weight - Sweat Tech Neoprene Hoodie or Jacket - Exercise Gear - For At Home or Gym Workouts
Silicone Pot Holders for Kitchen, 100% Cotton, Heat-Resistant Up to 500 Degrees, 1 Pack Titanium All-Clad Textiles
iPrimio 100% Natural Wool Eco-Friendly Cat & Kitten Cave Bed - Cozy House Indoor Bed for Cats & Kittens - Pet Felt Cat Cave, Cushion, Cove, Nest, Hideout, Hideaway, Tent, Tunnel Beds (Furry Pink)
CAVEPOP 5x7 Black Picture Frames 5-Sets, Made to Display (6.25x8.25” Ivory Color Mat 5x7”) Collage Picture Frame Sets
Amazon Brand – Pinzon Plush Cotton Throw - Burnt Orange
Sigma 18-35mm F1.8 Art DC HSM Lens for Pentax
ROCKURWOK Omelett

## Aplicação do KNN

### Tratamento e treino

In [73]:
dfMerged.head()

,ID_USUARIO,AVALIACAO,TITULO
22,AH5XW367PAIEXZ5DHBAA3YJLW66A,4.0,HIDBEA Frosted Window Privacy Film Non-Adhesiv...
35,AE2ZULCTYSG32NA6LETHNZ5U7LLA,5.0,Nylon Laundry Bag - Locking Drawstring Closure...
36,AH6USDXY54SVMNKRUYZQYEKTR6YA,5.0,NEWCOSPLAY Super Soft Throw Blanket Premium Si...
47,AFLLPGLN4S4QSMLDLSVFGD6AVHQQ,5.0,Corsair Vengeance Performance Memory Kit 32GB ...
57,AEUIPMMB4HKZML6LKKQA3XEHZMMA,4.0,DATA COMM Electronics 45-6001-WH-S 1-Gang Rece...


In [53]:
pivot.shape

(6086, 68199)

In [ ]:
pivot_filled = pivot.fillna(0)
# Utilização de matriz sparsa, compacta uma matriz com muitos zeros, evitando repetições.
sparse = csr_matrix(pivot_filled)

In [55]:
#Tipo do objeto
type(sparse)

scipy.sparse._csr.csr_matrix

In [75]:
sparse

<651x10000 sparse matrix of type '<class 'numpy.float32'>'
	with 4370 stored elements in Compressed Sparse Row format>

In [ ]:
# Criando e e treinando o modelo

model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(sparse)

NearestNeighbors(algorithm='brute')

### Criação da máquina preditiva

In [66]:
# Print da lista com as recomendações
def recommendationWithColaborativeKNN(product, num_of_elements=4):
    
    try:
        if product is None or product.strip() == "":
            raise ValueError("Forneça um nome de produto válido.")
        
        if(product not in pivot_filled.index):
            print(f"Produto '{product}' não encontrado.")
            return
        
        if (pivot_filled.loc[product] > 0).sum() < 4 or (pivot_filled.loc[product] == 0).sum() < num_of_elements:
            print("Produto com poucas avaliações, a recomendação pode não ser precisa.")
        
        # neighbors+1 pois o próprio produto será incluído na lista de vizinhos
        distance, sugestions = model.kneighbors(pivot_filled.filter(items = [product], axis=0).values.reshape(1, -1), n_neighbors=num_of_elements + 1) 
        # descomentar para ver a distância das recomendações
        # print(distance)
        
        print(f"Produto escolhido: '{product}'\n")
        print(f"Top {num_of_elements} recomendações:")
        for i in range(len(sugestions[0])):
            if pivot_filled.index[sugestions[0][i]] == product:
                continue
                
            print(pivot_filled.index[sugestions[0][i]])
            # print(f"Similaridade: {1 / (1 + distance[0][i]):.4f}\n")
    except KeyError as e:
        print(e)

### Funcionamento

In [70]:
searchProductName(input('Pesquise aqui o produto:'))

Star Wars Science Force Glove
LEGO Star Wars Poe Dameron's X-Wing Fighter 75273 Building Kit, Cool Construction Toy for Kids, New 2020 (761 Pieces)
Amazon Exclusive Star Wars The Child Chia Pet Floating Edition with Stand, “aka Baby Yoda” with Seed Packet, Decorative Pottery Planter, Easy to Do and Fun to Grow


In [85]:
recommendationWithColaborativeKNN(input('Entre com um produto que tenha gostado: '), 10) 

Produto escolhido: 'Pool Thermometer, INKBIRD Wireless Pool Thermometer Floating Easy Read, Pool Thermometer for Swimming Pool Hot Tubs Pond Test Kits & Thermometers 2nd Updated Generation'

Top 10 recomendações:
Power Queen 12V 200Ah Plus LiFePO4 Battery, 2560Wh Lithium Battery Built-in 200A BMS,over 4000 Cycles, Backup Battery in Case of Power Outage, Perfect for RV, Off-Grid System, Solar
V-MORO Compatible with Airpods Pro Case Leather, Genuine Leather Airpod Pro Case for Airpods Pro Protective Cover Skin Men Women Front LED Visible - Brown
DA VINCI 50 Clay Composite Dice Striped 11.5 Gram Poker Chips, Choose from 11 Colors
Gardzen 2 Uses Lawn Aerator Set, Plug Aeration & Spike Aerator, Heavy Duty Aerator for Compacted Soils and Lawns, 35" x 11", Black
farexon 55 x 24 Inch Electric Standing Desk Adjustable Height, Sit-Stand Desk with Oversized Mouse Pad， Double Crossbeam Structure, Four Preset Heights, 27''-45'' Lifting Range Stand up Desk
Galaxy S10+ Plus Case,Samsung S10+ Plus Cov